# Sayou Assembler Quick Start

**Sayou Fabric**의 조립 공장인 `sayou-assembler`를 시연합니다.
이 라이브러리는 Wrapper에서 표준화된 지식 노드(`SayouNode`)를 받아, 실제 데이터베이스(Graph DB, Vector DB)가 이해할 수 있는 형태(`Payload` 또는 `Query`)로 변환합니다.

**핵심 기능:**
1. **Graph Assembly:** 노드와 엣지 리스트로 변환 (역방향 엣지 자동 생성)
2. **Vector Assembly:** 텍스트를 임베딩 벡터로 변환 (검색용)
3. **Query Generation:** Neo4j Cypher 쿼리 등 DB 전용 언어로 변환

In [ ]:
import logging
from typing import List

from sayou.assembler.pipeline import AssemblerPipeline

logging.basicConfig(level=logging.INFO, format='%(message)s')

print("Import Successful!")

## 1. 파이프라인 초기화 & 임베딩 함수 주입

VectorBuilder가 작동하려면 텍스트를 숫자로 바꾸는 `embedding_fn`이 필요합니다.
여기서는 실제 API 대신, 텍스트 길이에 기반한 **Mock 함수**를 주입하여 시연합니다.

In [ ]:
def mock_embedding(text: str) -> List[float]:
    """(Mock) 텍스트 길이 기반 더미 벡터 생성"""
    val = float(len(text))
    return [val, val * 0.1, val * 0.2]

pipeline = AssemblerPipeline()
pipeline.initialize(embedding_fn=mock_embedding)

## 2. 입력 데이터 준비

`sayou-wrapper`가 출력했다고 가정한 표준 데이터(`WrapperOutput`)를 준비합니다.
(부모 노드 1개와 자식 노드 1개)

In [ ]:
wrapper_output = {
    "metadata": {"source": "demo.pdf"},
    "nodes": [
        {
            "node_id": "sayou:doc:parent_1",
            "node_class": "sayou:Topic",
            "friendly_name": "Introduction",
            "attributes": {"schema:text": "This is the Header"},
            "relationships": {} 
        },
        {
            "node_id": "sayou:doc:child_1",
            "node_class": "sayou:TextFragment",
            "attributes": {"schema:text": "This is the content paragraph."},
            "relationships": {
                "sayou:hasParent": ["sayou:doc:parent_1"]
            }
        }
    ]
}
print(f"Prepared {len(wrapper_output['nodes'])} nodes.")

## 3. 시나리오 1: Graph Assembly (Neo4j/NetworkX용)

`strategy="graph"`를 사용하면 **Nodes**와 **Edges** 리스트를 반환합니다.
특히, `hasParent`만 입력했지만 자동으로 `hasChild`(역방향) 엣지가 생성되는 것을 확인하세요.

In [ ]:
print(">>> Running Graph Builder...")
graph_data = pipeline.run(wrapper_output, strategy="graph")

print(f"Nodes: {len(graph_data['nodes'])}")
print(f"Edges: {len(graph_data['edges'])}")

print("\n--- Edge List (Forward & Reverse) ---")
for edge in graph_data['edges']:
    direction = "<--" if edge.get('is_reverse') else "-->"
    print(f"{edge['source']} {direction} [{edge['type']}] {direction} {edge['target']}")

## 4. 시나리오 2: Vector Assembly (Pinecone/Chroma용)

`strategy="vector"`를 사용하면 `id`, `vector`, `metadata`가 포함된 페이로드 리스트를 반환합니다.

In [ ]:
print(">>> Running Vector Builder...")
vector_payloads = pipeline.run(wrapper_output, strategy="vector")

for payload in vector_payloads:
    print(f"ID: {payload['id']}")
    print(f"Vector: {payload['vector']}")
    print(f"Meta: {list(payload['metadata'].keys())}")
    print("-" * 20)

## 5. 시나리오 3: Cypher Query Generation

`strategy="cypher"`를 사용하면 Neo4j에 바로 실행 가능한 `MERGE` 쿼리문들을 생성합니다.

In [ ]:
from sayou.assembler.plugins.cypher_builder import CypherBuilder

cypher_builder = CypherBuilder()
pipeline = AssemblerPipeline(extra_builders=[cypher_builder])
pipeline.initialize()

In [ ]:
print(">>> Running Cypher Builder...")
queries = pipeline.run(wrapper_output, strategy="cypher")

print(f"Generated {len(queries)} queries.\n")
for q in queries:
    print(q)